# Welcome to the Qualification User Tool for the RAPIDS Accelerator for Apache Spark
To run the user tool, you need to enter a log path that represents the DBFS location for your Spark CPU event logs.  Then you can select "Run all" to execute the notebook.  After the notebook completes, you will see various output tables show up below.  More options for running the qualification user tool can be found here: https://docs.nvidia.com/spark-rapids/user-guide/latest/spark-qualification-tool.html#running-the-qualification-tool-standalone-for-csp-environments-on-spark-event-logs.

## Summary Output
The report represents the entire app execution, including unsupported operators and non-SQL operations.  By default, the applications and queries are sorted in descending order by the following fields:
- Recommendation;
- Estimated GPU Speed-up;
- Estimated GPU Time Saved; and
- End Time.

## Stages Output
For each stage used in SQL operations, the Qualification tool generates the following information:
1. App ID
1. Stage ID
1. Average Speedup Factor: the average estimated speed-up of all the operators in the given stage.
1. Stage Task Duration: amount of time spent in tasks of SQL Dataframe operations for the given stage.
1. Unsupported Task Duration: sum of task durations for the unsupported operators. For more details, see Supported Operators.
1. Stage Estimated: True or False indicates if we had to estimate the stage duration.

## Execs Output
The Qualification tool generates a report of the “Exec” in the “SparkPlan” or “Executor Nodes” along with the estimated acceleration on the GPU. Please refer to the Supported Operators guide for more details on limitations on UDFs and unsupported operators.
1. App ID
1. SQL ID
1. Exec Name: example Filter, HashAggregate
1. Expression Name
1. Task Speedup Factor: it is simply the average acceleration of the operators based on the original CPU duration of the operator divided by the GPU duration. The tool uses historical queries and benchmarks to estimate a speed-up at an individual operator level to calculate how much a specific operator would accelerate on GPU.
1. Exec Duration: wall-Clock time measured since the operator starts till it is completed.
1. SQL Node Id
1. Exec Is Supported: whether the Exec is supported by RAPIDS or not. Please refer to the Supported Operators section.
1. Exec Stages: an array of stage IDs
1. Exec Children
1. Exec Children Node Ids
1. Exec Should Remove: whether the Op is removed from the migrated plan.

In [0]:
pip install spark-rapids-user-tools > /dev/null

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython() 

In [0]:
import pandas as pd
dbutils.widgets.text("log_path", "")
eventlog_string=dbutils.widgets.get("log_path")

dbutils.widgets.text("output_path", "")
outputpath_string=dbutils.widgets.get("output_path")

dbutils.widgets.dropdown("csp", "aws", ["aws", "azure"])
csp_string=dbutils.widgets.get("csp")
platform_string=f"databricks-{csp_string}"

In [0]:
!spark_rapids_user_tools $platform_string qualification --eventlogs $eventlog_string --local_folder $outputpath_string > /tmp/qual_debug.log

Processing...⣽

## Summary Output

In [0]:
summary_output=pd.read_csv(outputpath_string + "/rapids_4_spark_qualification_output/rapids_4_spark_qualification_output.csv")
display(summary_output)

App Name,App ID,Recommendation,Estimated GPU Speedup,Estimated GPU Duration,Estimated GPU Time Saved,SQL DF Duration,SQL Dataframe Task Duration,App Duration,GPU Opportunity,Executor CPU Time Percent,SQL Ids with Failures,Unsupported Read File Formats and Types,Unsupported Write Data Format,Complex Types,Nested Complex Types,Potential Problems,Longest SQL Duration,NONSQL Task Duration Plus Overhead,Unsupported Task Duration,Supported SQL DF Task Duration,Task Speedup Factor,App Duration Estimated,Unsupported Execs,Unsupported Expressions,Estimated Job Frequency (monthly)
TPC-DS Like Bench q4,app-20220209224316-0007,Strongly Recommended,2.78,40078.11,71498.88,90167,85324700,111577,90167,65.09,null,null,null,null,null,null,90163,3086516,0,85324700,4.83,false,Execute CreateViewCommand,null,30


## Stages Output

In [0]:
stages_output=pd.read_csv(outputpath_string + "/rapids_4_spark_qualification_output/rapids_4_spark_qualification_output_stages.csv")
display(stages_output)

App ID,Stage ID,Average Speedup Factor,Stage Task Duration,Unsupported Task Duration,Stage Estimated,Number of transitions from or to GPU
app-20220209224316-0007,45,5.91,13349286,0,false,0
app-20220209224316-0007,36,4.13,805680,0,false,0
app-20220209224316-0007,42,4.13,393456,0,false,0
app-20220209224316-0007,48,4.74,487756,0,false,0
app-20220209224316-0007,41,5.91,9850301,0,false,0
app-20220209224316-0007,31,3.76,1186,0,false,0
app-20220209224316-0007,49,4.75,567023,0,false,0
app-20220209224316-0007,40,4.13,733210,0,false,0
app-20220209224316-0007,32,3.76,707,0,false,0
app-20220209224316-0007,52,8.07,2747585,0,false,0


## Execs Output

In [0]:
execs_output=pd.read_csv(outputpath_string + "/rapids_4_spark_qualification_output/rapids_4_spark_qualification_output_execs.csv")
display(execs_output)

App ID,SQL ID,Exec Name,Expression Name,Task Speedup Factor,Exec Duration,SQL Node Id,Exec Is Supported,Exec Stages,Exec Children,Exec Children Node Ids,Exec Should Remove
app-20220209224316-0007,24,SortMergeJoin,null,22.33,0,101,true,45,null,null,false
app-20220209224316-0007,24,WholeStageCodegen (14),WholeStageCodegen (14),7.49,18790937,61,true,35,HashAggregate:Project:BroadcastHashJoin:Project:SortMergeJoin,62:63:64:65:66,false
app-20220209224316-0007,24,SubqueryBroadcast,null,3.0,976,43,true,null,null,null,false
app-20220209224316-0007,24,Exchange,null,4.9,12696,160,true,50:52,null,null,false
app-20220209224316-0007,24,WholeStageCodegen (2),WholeStageCodegen (2),6.64,17508276,29,true,39,Sort,30,false
app-20220209224316-0007,24,HashAggregate,null,4.41,0,24,true,39,null,null,false
app-20220209224316-0007,24,Sort,null,6.64,0,113,true,52,null,null,false
app-20220209224316-0007,24,WholeStageCodegen (10),WholeStageCodegen (10),6.64,18772156,67,true,35,Sort,68,false
app-20220209224316-0007,24,ReusedExchange,null,1.0,0,77,true,null,null,null,true
app-20220209224316-0007,24,HashAggregate,null,4.41,0,165,true,37,null,null,false
